# Using Latent Semantic Analysis for Code Comparison : Solutions to the Check for Palindromes Challenge

In this project I applied Latent Semantic Analysis to evaluate how close the solutions given by FCC students was. The compared solutions were for the "Check for Palindromes" problem and all were found in the FreeCodeCamp Torrent Dataset.

The code is a quick-and-dirty implementation.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


Data preparation consisted in a overall parsing of the codes some we could find the basics. In this project the parsing was a hard-coded implementation. Some other libraries and basic global variables were here also defined.

In [2]:
#!/usr/bin/env python3
## TEST!!!
import os, sys
import json
import re
import random
import queue
from collections import defaultdict, deque
import numpy
import pandas
import nltk
import matplotlib.pyplot as plt

The following section is about:
* obtaining the Torrent Dataset (in a local directory)
* selecting the code to be analysed
* parsing the code, including the elimination of undesirable lines (eg. comments, console.logs) and the renaming of some code elements (eg. variable names) into a single form that becomes comparable
* updating a dictionary with relevant information about the data

In [3]:

# global functionalities and variables
directory = "/add/your/path/here"

count = -1
data = []
data2 = defaultdict(int)
comp1 = re.compile(r"//(.*)\n") #delete comments
comp2 = re.compile(r"console\.log\([^\)]+\)") #delete console.logs
comp3 = re.compile(r"(\w+)")
comp4 = re.compile(r"function palindrome  \(  [^\)]+  \)")

indexid = -1
finaldata = defaultdict(dict)
usedcharacters = set()
with open(directory+"/output.json", "r") as f_in:
    while 1:
        record = f_in.readline()
        indexid += 1
        if not record:
            break
        if len(record) > 3:
            try:
                recordjson = json.loads(record[:-2])
            except ValueError:
                if record == '': continue
            for subrecord in recordjson:
                try:
                    #Check for Palindromes, Roman Numeral Converter
                    if "name" in list(subrecord.keys()):
                        data2[subrecord["name"]] = data2[subrecord["name"]] + 1
                        if subrecord["name"] == "Check for Palindromes":
                            #DATA CLEANING!!!
                            #data.append(subrecord["solution"])
                            #find regex?
                            modrec = " ".join([ x for x in re.sub(comp1," ", subrecord["solution"], re.M|re.I).replace("\n", " ").replace("\t", " ").split(" ") if x != "" ]).replace(";"," ")
                            modrec = re.sub(comp2, " ", modrec, re.M|re.I)

                            
                            solworegexq = deque(modrec.replace("\n", " ").replace("\t", " ").split("(")[::-1])
                            solworegex = []
                            regexlist = []
                            #print(solworegexq)
                            while len(solworegexq) > 0:
                                lookingforreplace = solworegexq.pop()
                                #print(lookingforreplace)
                                solworegex.append(lookingforreplace)
                                if "replace" in lookingforreplace.split(".") or "split" in lookingforreplace.split(".") or "match" in lookingforreplace.split(".") or "=" in lookingforreplace.split("."):
                                    count = 1
                                    regexsection = ''
                                    while count == 1:
                                        regexsection = solworegexq.pop()
                                        regexsectionsearch = re.search(r"/.+/[a-z]*", regexsection.replace('"', "'"))
                                        if regexsectionsearch.group(0):
                                            regexsec = regexsectionsearch.group(0)
                                            regexlist.append(regexsec)
                                            solworegex.append(regexsection.replace(regexsec, "REGEX"))
                                            
                                            count = 0
                                        else:
                                            solworegex.append(regexsection)
                            modrec = "(".join(solworegex)
                            modrec = " ".join([x for x in re.sub(comp3, r" \1 ", modrec, re.M|re.I).split(" ") if x != ""])                            
                            for r in ["(",")",".",",",":","=","{","}","==", "===","[","]","+","++","-","--","<",">","'",'"',"!","!=","?"]:
                                modrec = modrec.replace(r, " "+r+" ")
                            #finding variable lists
                            varlist = []
                            #first variables: arguments
                            #print(re.findall(comp4, modrec).replace(" ","").split("(")[1].split(")")[0])
                            varargs = re.search(comp4, modrec).group(0).replace(" ","").split("(")[1].split(")")[0]
                            for varname in varargs.split(","):
                                varlist.append(varname)
                            #function variables
                            modrecq = deque(modrec.split(" ")[::-1])
                            while len(modrecq) > 0:
                                lookingforvar = modrecq.pop()
                                if lookingforvar == "var":
                                    count = 1
                                    while count and len(modrecq) > 0:
                                        varname = modrecq.pop()
                                        if len(varname) > 0:
                                            varlist.append(varname)
                                            count = 0
                            ###print(varlist)
                            for ii, nname in enumerate(varlist):
                                modrec = modrec.replace(" "+nname+" ", " VAR_"+str(ii)+" ")
                            
                            #finally, remove last palindrome call
                            modrec = " ".join(modrec.split(" ")[:modrec.split(" ")[2:].index("palindrome") + 2])
                            ###print(modrec)
                            
                            data.append(modrec)
                            codewordcount = defaultdict(int)
                            for w in modrec.split(" "):
                                if w == "''": continue
                                codewordcount[w] = codewordcount[w] + 1
                            finaldata[indexid] = [modrec, codewordcount, subrecord["solution"], subrecord["completedDate"], regexlist, varlist]
                            usedcharacters = usedcharacters | set(codewordcount.keys())
                            
                except:
                    pass
                    #print("error in name")


(OBS: possible errors where silently ignored in this presentation, see except except above)

The target file was `finaldata`. That contains:
* *solution index*: according to the position index at which the solution was found in the Torrent Dataset
* *parsed solution*
* dict with the *characters count in the **parsed** solution* but the empty character - be aware that for a later work, the counts per solution were *standardised*
* *actual solution*
* *list of regex* that my parsing method managed to extract
* *list of variable names* that my parsing method managed to extract

After getting all the data in a more useful format I converted that file into pandas.



In [4]:
#build the matrix of list of lists (=records)
usedcharacters = sorted(usedcharacters)[1:] #eliminate the non-character character
datatopandas = []
s_solindex = sorted(finaldata)
for rec in s_solindex:                          #iterate through RECORDS (solutions given by campers for the palindrome problem)
    numcol = len(usedcharacters)
    rectopandas = [0]*numcol
    #rectopandas[0] = rec
    for char in list(finaldata[rec][1].keys()): #iterate through keys of characters counts per RECORD (= solution given by camper)
        if char == '' or char not in usedcharacters:                          #donot include the non-character character
            continue
        ichar = usedcharacters.index(char)
        rectopandas[ichar] = finaldata[rec][1][char]
    rowsum = sum(rectopandas)                    #prepare row in normalised form
    rectopandas = map(lambda x: x/rowsum, rectopandas)
    datatopandas.append(rectopandas)
        
char_df = pandas.DataFrame(datatopandas, columns = usedcharacters)
char_df_trans = char_df.transpose()



The pandas format consists in rows of parsed solutions against columns of the symbols and naming found in that solution.

In our target dataset we found 139 key symbols and names (rows) vs 153 solutions (columns):

In [5]:
char_df_trans.shape

(139, 153)

##LSA

In this part of the project I was using http://www.frankcleary.com/svd/ as main reference for the code.

In my case I had to made a transpose of the char_df pandas DataFrame just because the way it was constructed, but the rest resembles the aforementioned code:
* using numpy linalg.svd for singular value decomposition of the now char_df_trans matrix.
* identification and additional formatting of the right-singular unitary matrix representing the parsed solutions
* analysis of the distances between parsed solutions by comparing column to column of the right-singular values (see `dist` function)

In the case below, I left the indexing of the solution to be its position in the DataFrame and **not in the Torrent Dataset** as previously defined.

In [7]:
U, sigma, V = numpy.linalg.svd(char_df_trans)
v_df = pandas.DataFrame(V)
v_df.apply(lambda x: numpy.round(x, decimals=2))


def dist(col1, col2, sigma=sigma):
    """Return the norm of (col1 - col2), where the differences in 
    each dimension are wighted by the values in sigma."""
    return numpy.linalg.norm(numpy.array(col1 - col2) * numpy.append(sigma,[0]*(len(dist_df)-len(sigma))))
    
dist_df = pandas.DataFrame(index=v_df.columns, columns=v_df.columns)
for sol in v_df.columns:
    dist_df[sol] = v_df.apply(lambda x: dist(v_df[sol].values, x.values))



The shape of the dist_df matrix (dataframe) gives us the number of solutions to be evaluated:  

In [8]:
dist_df.shape

(153, 153)

Then, following the method suggested by Frank (above) I made a heatmap of the distances between solutions, in between three-ranking colouring.

In [10]:
#levels = [0.06, 0.075]
levels = [.24, 0.48]
binned_df = dist_df.copy()
binned_df[(dist_df <= levels[0]) & (dist_df > 0)] = 1
binned_df[(dist_df <= levels[1]) & (dist_df > levels[0])] = 2
binned_df[(dist_df < 1) & (dist_df > levels[1])] = 3
plt.imshow(binned_df.values, interpolation='none')
ax = plt.gca()
plt.xticks(range(0, len(binned_df.columns.values), 30))
plt.yticks(range(0, len(binned_df.index.values), 30))
labels = [x for x in range(0, len(binned_df.columns.values), 30) if x%30 == 0]
ax.set_xticklabels(labels, rotation=90)
ax.set_yticklabels(labels)
plt.title("Similarity between solutions\nLower value = more similar")
plt.colorbar()
plt.show()

Notice that the numeration do NOT correspond to the name given to the solution but to the order that they took in the matrix.


Finally, we make a comparison, based on the actual indexing of the solution as in the Torrent Dataset:

In [11]:
#for sol in s_solindex:
for sol in range(10):
    sim_sols_df = dist_df.sort(columns=sol)[sol]
    sim_sols = sim_sols_df.drop([sol]).index
    print('10 first solutions most similar to ' + str(s_solindex[sol]) + ':')
    sss = []
    for ss in sim_sols[:10]:
        sss.append(s_solindex[ss])
    print(sss)
    print('\n')


10 first solutions most similar to 22:
[4070, 3440, 3415, 1783, 2417, 3474, 204, 2810, 5674, 1112]


10 first solutions most similar to 60:
[2427, 3368, 2465, 3687, 429, 337, 3420, 5270, 294, 1411]


10 first solutions most similar to 87:
[727, 3640, 1315, 5018, 347, 1209, 2576, 4976, 337, 2310]


10 first solutions most similar to 118:
[2576, 1209, 1612, 4561, 3295, 3543, 1851, 4433, 1315, 2479]


10 first solutions most similar to 204:
[2810, 1873, 1112, 4691, 1783, 22, 3141, 1845, 4433, 2417]


10 first solutions most similar to 209:
[2254, 1290, 2647, 2944, 2663, 1414, 651, 5558, 2334, 3389]


10 first solutions most similar to 294:
[3171, 3443, 337, 1315, 3580, 429, 3848, 2417, 87, 727]


10 first solutions most similar to 337:
[429, 5674, 2093, 881, 1411, 3420, 3474, 3443, 1315, 2417]


10 first solutions most similar to 347:
[727, 87, 2948, 1377, 3640, 2311, 5018, 4976, 1181, 3153]


10 first solutions most similar to 397:
[2274, 3687, 60, 1501, 2465, 3084, 3602, 3295, 1411, 122